In [1]:
!pip install llama-index llama-index-agent-openai llama-index-llms-openai llama-index-readers-file llama-index-embeddings-openai pymupdf gradio

In [2]:
## 필요한 도구 임포트
import os
import requests

# 라마인덱스 관련 임포트
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, PromptTemplate, Settings
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.agent import ReActAgent

In [3]:
## OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = "여러분의 키 값"

## 실습 데이터 다운로드
urls = [
    "https://raw.githubusercontent.com/AgnetHub/LlamaIndex_Code/main/ch06/data/autonomous_car_technology.pdf",
    "https://raw.githubusercontent.com/AgnetHub/LlamaIndex_Code/main/ch06/data/ict_market_trends_autonomous_driving_2024.pdf"
]

In [4]:
# 각 파일 다운로드
for url in urls:
    filename = url.split("/")[-1]  # URL에서 파일명 추출
    response = requests.get(url)

    with open(filename, "wb") as f:
        f.write(response.content)
    print(f"{filename} 다운로드 완료")

autonomous_car_technology.pdf 다운로드 완료
ict_market_trends_autonomous_driving_2024.pdf 다운로드 완료


In [5]:
## 라마인덱스 Settings 설정
# LLM, 임베딩 모델, 노드 파서를 포함하는 전역 설정
Settings.llm = OpenAI(model="gpt-4o-2024-08-06", temperature=0)
Settings.embed_model = OpenAIEmbedding()
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=0)

In [6]:
def create_pdf_retriever(pdf_path, persist_directory, similarity_top_k=5):
    """
    PDF 파일을 벡터 데이터베이스로 변환하고 검색 엔진을 반환하는 함수

    Args:
        pdf_path (str): PDF 파일 경로
        persist_directory (str): 벡터 저장소 경로
        similarity_top_k (int): 검색 시 반환할 상위 결과 개수

    Returns:
        query_engine: 검색 엔진 객체
    """
    # PDF 파일 로드
    documents = SimpleDirectoryReader(input_files=[pdf_path]).load_data()
    print(f"{pdf_path}에서 로드된 총 문서 수: {len(documents)}")

    # 벡터 인덱스 생성
    index = VectorStoreIndex.from_documents(documents, embed_model=Settings.embed_model)
    print(f"{pdf_path} 인덱스에 있는 노드 수: {len(index.docstore.docs)}")

    # 필요시 인덱스를 디스크에 저장
    index.storage_context.persist(persist_dir=persist_directory)

    # 검색 엔진 생성 - 심플하게 similarity_top_k만 설정
    engine = index.as_query_engine(similarity_top_k=similarity_top_k)

    return engine

In [7]:
# 미국 ICT 정책 데이터베이스 생성
retriever_autonomous_car_technology = create_pdf_retriever(
    pdf_path="autonomous_car_technology.pdf",
    similarity_top_k=5,
    persist_directory="db_autonomous_car_technology"
)

autonomous_car_technology.pdf에서 로드된 총 문서 수: 7
autonomous_car_technology.pdf 인덱스에 있는 노드 수: 34


In [8]:
# 일본 ICT 정책 데이터베이스 생성
retriever_ict_market_autonomous_car = create_pdf_retriever(
    pdf_path="ict_market_trends_autonomous_driving_2024.pdf",
    similarity_top_k=5,
    persist_directory="db_ict_market_trends_autonomous_driving_2024"
)

ict_market_trends_autonomous_driving_2024.pdf에서 로드된 총 문서 수: 29
ict_market_trends_autonomous_driving_2024.pdf 인덱스에 있는 노드 수: 68


In [9]:
# 검색 도구 생성
autonomous_car_technology_engine = QueryEngineTool(
    query_engine=retriever_autonomous_car_technology,
    metadata=ToolMetadata(
        name="autonomous_car_technology",
        description="자율주행차의 기술 정보를 제공합니다. 자율주행차의 기술 관련된 질문은 반드시 이 도구를 사용합니다.",
    ),
)

ict_market_autonomous_car_engine = QueryEngineTool(
    query_engine=retriever_ict_market_autonomous_car,
    metadata=ToolMetadata(
        name="ict_market_autonomous_car",
        description="자율주행차의 ICT 시장동향 정보를 제공합니다. 자율주행차의 ICT와 시장 동향과 관련된 질문은 반드시 이 도구를 사용합니다.",
    ),
)

In [10]:
# 도구 리스트 생성
tools = [autonomous_car_technology_engine, ict_market_autonomous_car_engine]

In [11]:
template = """\
당신은 질문에 답변하는 것부터 요약 제공, 기타 여러 유형의 분석까지 다양한 작업을 돕기 위해 설계되었습니다.

## 도구
당신은 다양한 도구에 접근할 수 있습니다. 현재 작업을 완료하기 위해 적절하다고 판단되는 순서로 도구를 사용하는 것은 당신의 책임입니다. 이를 위해 작업을 하위 작업으로 나누고, 각 하위 작업에 다른 도구를 사용할 필요가 있을 수 있습니다.

당신은 다음 도구들에 접근할 수 있습니다:
{tool_desc}

## 출력 형식
질문에 답변하기 위해 다음 형식을 사용하십시오.

###
Thought: I need to use a tool to help me answer the question.
Action: 도구 이름 (사용할 도구 중 하나인 {tool_names})
Action Input: 도구에 대한 입력을 JSON 형식으로 제공하십시오. 예: {{\"input\": \"hello world\", \"num_beams\": 5}}
###

항상 'Thought'로 시작하십시오.

'Action Input'에서는 올바른 JSON 형식을 사용하십시오. 이렇게 쓰지 마십시오: {{'input': 'hello world', 'num_beams': 5}}.

이 형식이 사용되면, 사용자는 다음 형식으로 응답할 것입니다:

###
Observation: 도구 응답
###

이 형식을 계속 반복하여 더 이상 도구를 사용하지 않고 질문에 답변할 수 있을 만큼 충분한 정보를 얻을 때까지 진행하십시오.
그 시점에서는 반드시 다음 두 가지 형식 중 하나로 응답해야 합니다:

###
Thought: I can answer without using any more tools.
Answer: [여기에 답변을 작성하세요]
###

###
Thought: I cannot answer the question with the provided tools.
Answer: 죄송합니다. 해당 질문에 답변할 수 없습니다.
###

## 추가 규칙
- 답변은 반드시 질문에 도달하기까지의 과정을 설명하는 순차적인 항목들로 구성되어야 합니다. 여기에는 이전 대화의 내용이 포함될 수 있습니다.
- 각 도구의 함수 서명을 반드시 준수해야 하며, 함수가 인수를 기대할 경우 인수를 생략하지 마십시오.
- 답변은 반드시 '한글'로 상세하게 작성되어야 합니다.
- 질문에 대한 답변만 작성하세요. 질문과 관계없는 검색을 수행하지 마십시오. 이는 매우 중요합니다.
- Please follow the thought-action-input format.
- 하나의 질문에 많은 검색어가 포함되어져 있는 것처럼 보인다면 검색어를 나누어서 순차적으로 검색하십시오. 더 좋은 답변을 얻을 수 있을 것입니다.
- 도구를 사용하여 답변할 수 있는 주제라면 반드시 도구를 사용하시기 바랍니다. 이는 매우 중요합니다.
- 당신이 도구없이 답변하는 것은 도구의 주제와 완전히 다른 주제의 질문이 들어왔을 때 뿐입니다. 도구와 연관된 질문이라면 반드시 도구를 호출하십시오. 이는 매우 중요하며 당신이 지켜야 할 1순위의 우선사항입니다.

## 현재 대화
아래는 인간과 어시스턴트 메시지가 교차되어 있는 현재 대화 내용입니다.

"""

In [12]:
## 5.2.4 에이전트 객체 생성
# ReAct 에이전트 생성
react_agent = ReActAgent.from_tools(
    tools=tools,
    verbose=True
)

/usr/local/lib/python3.12/dist-packages/llama_index/core/agent/react/base.py:154: DeprecationWarning: Call to deprecated class ReActAgent. (ReActAgent has been rewritten and replaced by llama_index.core.agent.workflow.ReActAgent.

This implementation will be removed in a v0.13.0 and the new implementation will be promoted to the `from llama_index.core.agent import ReActAgent` path.

See the docs for more information: https://docs.llamaindex.ai/en/stable/understanding/agent/)
  return cls(
/usr/local/lib/python3.12/dist-packages/deprecated/classic.py:184: DeprecationWarning: Call to deprecated class AgentRunner. (AgentRunner has been deprecated and is not maintained.

This implementation will be removed in a v0.13.0.

See the docs for more information on updated agent usage: https://docs.llamaindex.ai/en/stable/understanding/agent/)
  return old_new1(cls, *args, **kwargs)


In [13]:
react_system_prompt = PromptTemplate(template)
react_agent.update_prompts({"agent_worker:system_prompt": react_system_prompt})
react_agent.reset()

In [14]:
result1 = react_agent.query("한국에서의 자율주행차의 시장 동향이 궁금해")
print("최종 답변:", result1.response)
print("\n" + "="*50 + "\n")

> Running step a57e6d99-75a0-4b71-9194-8d9624de785c. Step input: 한국에서의 자율주행차의 시장 동향이 궁금해
Thought: ###
Thought: I need to use a tool to help me answer the question about the market trends of autonomous cars in South Korea.
Action: ict_market_autonomous_car
Action Input: {'input': '한국에서의 자율주행차 시장 동향'}
Observation: 한국의 자율주행차 시장은 2022년부터 2026년까지 꾸준한 성장을 보일 것으로 예상됩니다. 2022년 시장 규모는 0.3십억 달러였으며, 2026년에는 0.74십억 달러에 이를 것으로 보입니다. 이러한 성장세는 자율주행차 기술의 발전과 관련 인프라의 확충에 따른 것으로 보입니다.
> Running step 787f7a84-5544-4dd4-8782-9b255b7ae3bb. Step input: None
Thought: I can answer without using any more tools.
Answer: 한국의 자율주행차 시장은 2022년부터 2026년까지 꾸준한 성장을 보일 것으로 예상됩니다. 2022년 시장 규모는 0.3십억 달러였으며, 2026년에는 0.74십억 달러에 이를 것으로 보입니다. 이러한 성장세는 자율주행차 기술의 발전과 관련 인프라의 확충에 따른 것으로 보입니다.
최종 답변: 한국의 자율주행차 시장은 2022년부터 2026년까지 꾸준한 성장을 보일 것으로 예상됩니다. 2022년 시장 규모는 0.3십억 달러였으며, 2026년에는 0.74십억 달러에 이를 것으로 보입니다. 이러한 성장세는 자율주행차 기술의 발전과 관련 인프라의 확충에 따른 것으로 보입니다.




In [15]:
print("한국에서의 자율주행차의 시장 동향은 어떤지, 그리고 요즘 자율주행차에서 핫한 기술에는 뭐가 있는지 궁금해")
result2 = react_agent.query("한국에서의 자율주행차의 시장 동향은 어떤지, 그리고 요즘 자율주행차에서 핫한 기술에는 뭐가 있는지 궁금해")
print("최종 답변:", result2.response)
print("\n" + "="*50 + "\n")

한국에서의 자율주행차의 시장 동향은 어떤지, 그리고 요즘 자율주행차에서 핫한 기술에는 뭐가 있는지 궁금해
> Running step 17bfbeb2-8d6d-4b5f-b51a-1b8190a0b408. Step input: 한국에서의 자율주행차의 시장 동향은 어떤지, 그리고 요즘 자율주행차에서 핫한 기술에는 뭐가 있는지 궁금해
Thought: I need to use a tool to help me answer the question about the market trends and technologies in autonomous cars in Korea.
Action: ict_market_autonomous_car
Action Input: {'input': '한국에서의 자율주행차 시장 동향'}
Observation: 한국의 자율주행차 시장은 2022년부터 2026년까지 꾸준한 성장을 보이고 있습니다. 2022년 시장 규모는 0.3십억 달러였으며, 2026년에는 0.74십억 달러에 이를 것으로 예상됩니다. 이러한 성장세는 자율주행차 기술의 발전과 관련 인프라의 확충에 따른 것으로 보입니다.
> Running step 67790d87-a5cf-44f7-981b-bf51a90c5ebe. Step input: None
Thought: I also need to find out about the latest hot technologies in autonomous cars.
Action: autonomous_car_technology
Action Input: {'input': '요즘 자율주행차에서 핫한 기술'}
Observation: 최근 자율주행차에서 주목받고 있는 기술로는 자율운전, 운전자 지원, 텔레매틱스 분야가 있습니다. 특히 자율운전 기술에 많은 특허가 집중되고 있으며, 이 분야에서 일본 기업들이 강세를 보이고 있습니다. 또한, 자율주행차의 핵심기술은 스마트카의 발전과 함께 자동차 산업에 큰 영향을 미칠 것으로 예상됩니다.
> Running step 77241f

In [16]:
# ReACT 에이전트 실행 출력을 캡처
import io
import sys
import gradio as gr
from contextlib import redirect_stdout

In [17]:
# 멀티턴 대화를 위한 챗봇 인터페이스 구현
class ChatbotWithMemory:
    def __init__(self, react_agent):
        """
        멀티턴 대화 기능을 갖춘 챗봇 초기화

        Args:
            react_agent: 초기화된 ReACT 에이전트
        """
        self.react_agent = react_agent
        # 대화 기록을 저장하기 위한 변수
        self.chat_history = []
        # 에이전트 출력을 캡처하기 위한 설정
        self.verbose_capture = True

    def reset(self):
        """대화 기록과 에이전트 상태 초기화"""
        self.chat_history = []
        # 에이전트 상태 초기화
        self.react_agent.reset()
        return [], ""

    def chat(self, message, history):
        """
        사용자 메시지에 응답하고 대화 기록 업데이트

        Args:
            message: 사용자의 현재 입력 메시지
            history: 그라디오가 관리하는 대화 기록

        Returns:
            튜플: (업데이트된 대화 기록, 대화 기록, 로그)
        """
        if message.strip() == "":
            return history, history, ""

        # 이전 대화 맥락을 고려한 프롬프트 생성
        context = self._build_context()

        # 사용자 메시지와 컨텍스트를 합쳐 쿼리 생성
        query = message
        if context:
            query = f"{context}\n\n현재 질문: {message}"

        # 출력 캡처 시작
        f = io.StringIO()
        with redirect_stdout(f):
            # 에이전트에 쿼리 전송 (verbose=True로 설정되어 있음)
            response = self.react_agent.query(query)

        # 캡처된 출력 가져오기
        execution_log = f.getvalue()

        # 대화 기록에 추가
        self.chat_history.append({"role": "user", "content": message})
        self.chat_history.append({"role": "assistant", "content": response.response})

        # 그라디오 대화 기록 업데이트
        history.append((message, response.response))

        # 로그 정보 포함
        log_info = f"에이전트에 전달된 쿼리:\n{query}\n\n실행 로그:\n{execution_log}"

        return history, history, log_info

    def _build_context(self):
        """이전 대화 기록을 바탕으로 컨텍스트 생성"""
        if not self.chat_history:
            return ""

        # 최대 3개의 이전 대화 턴을 컨텍스트로 사용
        recent_turns = self.chat_history[-6:] if len(self.chat_history) > 6 else self.chat_history
        context_parts = []

        for i in range(0, len(recent_turns), 2):
            if i+1 < len(recent_turns):
                user_msg = recent_turns[i]["content"]
                assistant_msg = recent_turns[i+1]["content"]
                context_parts.append(f"사용자: {user_msg}\n에이전트: {assistant_msg}")

        if context_parts:
            return "이전 대화 내용:\n" + "\n\n".join(context_parts)
        return ""

In [18]:
# 그라디오 인터페이스 구현
def create_gradio_interface():
    # 챗봇 객체 생성
    chatbot = ChatbotWithMemory(react_agent)

    # 그라디오 인터페이스 구성
    with gr.Blocks(title="자율주행차 ReACT 에이전트", theme=gr.themes.Soft()) as demo:
        gr.Markdown("# 자율주행차 정보 검색 AI 비서")
        gr.Markdown("자율주행차의 기술과 시장 동향에 관한 질문을 해보세요. 멀티턴 대화가 가능합니다.")

        with gr.Row():
            with gr.Column(scale=7):
                chatbot_interface = gr.Chatbot(
                    height=500,
                    show_label=False,
                )

                with gr.Row():
                    msg_input = gr.Textbox(
                        placeholder="질문을 입력하세요...",
                        label="메시지",
                        scale=8
                    )
                    submit_btn = gr.Button("전송", scale=1)
                    clear_btn = gr.Button("초기화", scale=1)

            # 오른쪽에 실행 로그 표시 영역 추가
            with gr.Column(scale=3):
                gr.Markdown("### 에이전트 실행 로그")
                log_output = gr.Textbox(
                    label="실행 로그",
                    lines=25,
                    max_lines=25,
                    interactive=False
                )

        # 이벤트 핸들러 등록
        submit_btn.click(
            fn=chatbot.chat,
            inputs=[msg_input, chatbot_interface],
            outputs=[chatbot_interface, chatbot_interface, log_output],
        ).then(
            fn=lambda: "",
            outputs=msg_input,
        )

        msg_input.submit(
            fn=chatbot.chat,
            inputs=[msg_input, chatbot_interface],
            outputs=[chatbot_interface, chatbot_interface, log_output],
        ).then(
            fn=lambda: "",
            outputs=msg_input,
        )

        clear_btn.click(
            fn=chatbot.reset,
            outputs=[chatbot_interface, log_output],
        )

    return demo

# 그라디오 앱 생성 (실행은 아래에서 별도로)
demo = create_gradio_interface()

# 그라디오 앱 실행
demo.launch(share=True)  # 필요시 주석 해제

/tmp/ipython-input-324648665.py:13: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_interface = gr.Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b687cf5422ca828361.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
